# 🧠 Persistence in LangGraph

## 📌 What is Persistence?

In LangGraph, **persistence** refers to the ability to **store and reload the graph state** between different executions.

This is essential for:
- Long-running workflows
- Error recovery
- Human in the loop
- Short term memory
- Time travel
- Stateful applications (e.g., agents that remember conversations)
- Distributed processing

Without persistence, all state is lost once execution stops.

---

## 💾 Why is Persistence Needed?

LangGraph maintains a **graph state** which tracks:
- Node outputs
- Variable values
- Control flow decisions

If you don’t persist this state:
- You cannot resume the graph from where it stopped
- You lose all memory/context of what happened before

---

## 🔧 How is Persistence Implemented?

LangGraph provides **persistence interfaces** through:

### 1. `StateStore`:
An interface for how states are stored.

Implementations include:
- `InMemoryStateStore`: Keeps everything in memory (good for testing)
- `RedisStateStore`: Uses Redis for scalable, fault-tolerant persistence
- `MongoDBStateStore`: Persists state in MongoDB
- Custom backends: You can define your own `StateStore`

### 2. `graph.persist()`
You call this method (or configure the graph) to make sure it writes state to the storage backend.

---

## ✅ Example

```python
from langgraph.graph import StateGraph
from langgraph.storage import RedisStateStore

store = RedisStateStore(url="redis://localhost:6379")

graph = StateGraph()
graph.configure(persistence=store)

```

Now:

- Every step's state will be written to Redis.
- If execution crashes or is paused, you can resume from last state.



## 🧵 What are Threads in LangGraph Persistence?

In LangGraph, a thread is a unique instance of a graph execution — like a session or conversation.

Each thread has:
- Its own state
- A unique thread_id
- A history of all updates and steps taken in the graph

In [1]:
from langgraph.graph import START, END, StateGraph
from typing import TypedDict
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

from langgraph.checkpoint.memory import InMemorySaver

In [2]:
class JokeState(TypedDict):
    topic : str
    joke : str
    explanation : str

In [3]:
llm = ChatOpenAI()

In [8]:
def generate_joke(state: JokeState):
    prompt = f"Write a funny joke on the topic {state['topic']}"
    response = llm.invoke(prompt).content
    return {'joke': response}

def generate_explanation(state:JokeState):
    prompt = f'write an explanation on the following joke: {state['joke']}'
    response1 = llm.invoke(prompt).content
    return{'explanation':response1}

In [9]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)

graph.add_node("generate_explanation", generate_explanation)

graph.add_edge(START,"generate_joke")
graph.add_edge('generate_joke', "generate_explanation")
graph.add_edge('generate_explanation', END)

In [10]:
checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer= checkpointer)

In [12]:
config1 = {'configurable':{'thread_id':'1'}}

workflow.invoke({'topic':"humans"}, config = config1)

{'topic': 'humans',
 'joke': 'Why did the human bring a ladder to the bar?\n\nBecause they heard the drinks were on the house!',
 'explanation': 'This joke plays on the double meaning of the phrase "on the house." In a bar or restaurant, when something is "on the house," it means that it is free of charge. However, in this joke, the human brings a ladder to the bar because they interpret "on the house" literally - as in, the drinks are positioned on top of the bar building. This misunderstanding adds humor to the situation and creates a playful and silly image of someone bringing a ladder to a bar to reach their free drinks.'}

In [13]:
workflow.get_state((config1))

StateSnapshot(values={'topic': 'humans', 'joke': 'Why did the human bring a ladder to the bar?\n\nBecause they heard the drinks were on the house!', 'explanation': 'This joke plays on the double meaning of the phrase "on the house." In a bar or restaurant, when something is "on the house," it means that it is free of charge. However, in this joke, the human brings a ladder to the bar because they interpret "on the house" literally - as in, the drinks are positioned on top of the bar building. This misunderstanding adds humor to the situation and creates a playful and silly image of someone bringing a ladder to a bar to reach their free drinks.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06fb66-b407-65b9-8006-de9fefaf23f7'}}, metadata={'source': 'loop', 'writes': {'generate_explanation': {'explanation': 'This joke plays on the double meaning of the phrase "on the house." In a bar or restaurant, when something is "on the house," it mean

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'humans', 'joke': 'Why did the human bring a ladder to the bar?\n\nBecause they heard the drinks were on the house!', 'explanation': 'This joke plays on the double meaning of the phrase "on the house." In a bar or restaurant, when something is "on the house," it means that it is free of charge. However, in this joke, the human brings a ladder to the bar because they interpret "on the house" literally - as in, the drinks are positioned on top of the bar building. This misunderstanding adds humor to the situation and creates a playful and silly image of someone bringing a ladder to a bar to reach their free drinks.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06fb66-b407-65b9-8006-de9fefaf23f7'}}, metadata={'source': 'loop', 'writes': {'generate_explanation': {'explanation': 'This joke plays on the double meaning of the phrase "on the house." In a bar or restaurant, when something is "on the house," it mea

In [15]:
config2 = {'configurable':{"thread_id":"2"}}

workflow.invoke({'topic':'cricket'}, config= config2)

{'topic': 'cricket',
 'joke': 'Why did the cricket team go to the library?\n\nBecause they wanted to improve their bowling skills!',
 'explanation': 'This joke plays on the word "bowling," which can refer to both the sport of cricket and the sport of bowling in a library. In cricket, bowling refers to the act of throwing the ball towards the batsman, while in a library, bowling refers to the act of rolling a ball down a lane to knock over pins.\n\nThe punchline of the joke suggests that the cricket team went to the library not to improve their cricket skills, which one might expect, but rather to improve their bowling skills in the sense of knocking down pins. This unexpected twist in the joke adds humor and a playful element of surprise for the audience.'}

In [18]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'cricket', 'joke': 'Why did the cricket team go to the library?\n\nBecause they wanted to improve their bowling skills!', 'explanation': 'This joke plays on the word "bowling," which can refer to both the sport of cricket and the sport of bowling in a library. In cricket, bowling refers to the act of throwing the ball towards the batsman, while in a library, bowling refers to the act of rolling a ball down a lane to knock over pins.\n\nThe punchline of the joke suggests that the cricket team went to the library not to improve their cricket skills, which one might expect, but rather to improve their bowling skills in the sense of knocking down pins. This unexpected twist in the joke adds humor and a playful element of surprise for the audience.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f06fb77-26dc-6198-8002-e0a2069dc834'}}, metadata={'source': 'loop', 'writes': {'generate_explanation': {'explanation': 